# OpenRouter request shape (OpenAI-compatible)

This notebook is for quickly testing the exact request payload + headers we send to OpenRouter, including the newer "reasoning effort" control (which is header-based for some models/providers).

## Env vars

- `OPENROUTER_API_KEY` (required)
- `OPENROUTER_BASE_URL` (optional, default: `https://openrouter.ai/api/v1`)
- `OPENROUTER_HTTP_REFERER` (optional)
- `OPENROUTER_TITLE` (optional)
- `OPENROUTER_TEST_MODEL` (optional, e.g. `google/gemini-2.0-flash-001`)
- `OPENROUTER_REASONING_EFFORT_HEADER` (optional, default: `X-Reasoning-Effort`)
- `OPENROUTER_REASONING_EFFORT` (optional: `low`|`medium`|`high`)


In [ ]:
%pip -q install openai python-dotenv

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()

OPENROUTER_BASE_URL = os.getenv("OPENROUTER_BASE_URL", "https://openrouter.ai/api/v1")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY") or os.getenv("OPENAI_API_KEY")
OPENROUTER_HTTP_REFERER = os.getenv("OPENROUTER_HTTP_REFERER", "")
OPENROUTER_TITLE = os.getenv("OPENROUTER_TITLE", "FateForger Notebook")

if not OPENROUTER_API_KEY:
    raise RuntimeError("Missing OPENROUTER_API_KEY (or OPENAI_API_KEY)")

default_headers = {"X-Title": OPENROUTER_TITLE}
if OPENROUTER_HTTP_REFERER:
    default_headers["HTTP-Referer"] = OPENROUTER_HTTP_REFERER

client = OpenAI(
    base_url=OPENROUTER_BASE_URL,
    api_key=OPENROUTER_API_KEY,
    default_headers=default_headers,
)

print("base_url:", OPENROUTER_BASE_URL)
print("has key:", bool(OPENROUTER_API_KEY))
print("default_headers:", default_headers)

In [ ]:
# Optional: list a few models to confirm IDs
models = client.models.list()
for m in models.data[:20]:
    print(m.id)

In [ ]:
MODEL = os.getenv("OPENROUTER_TEST_MODEL", "google/gemini-2.0-flash-001")
print("MODEL:", MODEL)

resp = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are concise and helpful."},
        {"role": "user", "content": "Say hi and ask me one question about my day."},
    ],
)

print(resp.choices[0].message.content)

In [ ]:
# Reasoning effort: OpenRouter supports multiple patterns depending on model/provider.
# - Body-based (OpenRouter-compatible): extra_body={"reasoning": {"effort": "high"}}
# - Header-based (some providers/models): extra_headers={"X-Reasoning-Effort": "high"}

effort = os.getenv("OPENROUTER_REASONING_EFFORT", "high")
effort_header_name = os.getenv("OPENROUTER_REASONING_EFFORT_HEADER", "X-Reasoning-Effort")
send_body = os.getenv("OPENROUTER_SEND_EFFORT_IN_BODY", "true").lower() == "true"
send_header = os.getenv("OPENROUTER_SEND_EFFORT_IN_HEADER", "false").lower() == "true"

kwargs = {}
if send_body:
    kwargs["extra_body"] = {"reasoning": {"effort": effort}}
if send_header:
    kwargs["extra_headers"] = {effort_header_name: effort}

resp = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Write a 6-bullet plan to clean up a messy backlog."},
    ],
    **kwargs,
)

print("effort:", effort)
print("send_body:", send_body, "send_header:", send_header)
print(resp.choices[0].message.content)